# データ加工編　No.2

In [122]:
# 最初に実行すること
# 該当ノートブックに必要ないものも含まれているので注意

import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import japanize_matplotlib

from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

from imblearn.under_sampling import RandomUnderSampler

In [123]:
warnings.filterwarnings('ignore')

# データの格納先が異なる場合は変更し、実行すること
df_customer = pd.read_csv('data/customer.csv')
df_category = pd.read_csv('data/category.csv')
df_product = pd.read_csv('data/product.csv')
df_receipt = pd.read_csv('data/receipt.csv')
df_store = pd.read_csv('data/store.csv')
df_geocode = pd.read_csv('data/geocode.csv')

## 課題 21〜40

---
21. レシート明細データ（df_receipt）に対し、件数をカウントしてください。

In [124]:
len(df_receipt)

104681

---
22. レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニークな件数をカウントしてください。

In [125]:
len(df_receipt['customer_id'].unique())

8307

---
23. レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計したものを表示してください。

In [126]:
df_receipt.groupby('store_cd')[['amount', 'quantity']].agg('sum').reset_index()
# df_receipt.groupby('store_cd')[['amount', 'quantity']].apply(lambda x: np.sum(x)).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
24. レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示してください。

In [127]:
df_receipt.groupby('customer_id').agg({'sales_ymd': 'max'}).reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
25. レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示してください。

In [128]:
df_receipt.groupby('customer_id').agg({'sales_ymd': 'min'}).reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
26. レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示してください。

In [129]:
tmp1 = df_receipt.groupby('customer_id')['sales_ymd'].max().reset_index()
tmp2 = df_receipt.groupby('customer_id')['sales_ymd'].min().reset_index()
tmp_df = pd.merge(tmp1, tmp2, on='customer_id', how='left')
tmp_df.query('sales_ymd_x != sales_ymd_y').head(10)
# tmp_df = df_receipt.groupby('customer_id').agg({'sales_ymd': ['max', 'min']}).reset_index()
# tmp_df.columns = ['customer_id', 'sales_ymd_max', 'sales_ymd_min']
# tmp_df.query('sales_ymd_max != sales_ymd_min').head(10)

,customer_id,sales_ymd_x,sales_ymd_y
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


---
27. レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、上位5件を表示してください。

In [130]:
# df_receipt.groupby('store_cd')['amount'].apply(lambda x: np.mean(x)).reset_index().sort_values('amount', ascending=False).head(5)
df_receipt.groupby('store_cd')['amount'].mean().reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,402.867470
12,S13015,351.111960
7,S13003,350.915519
30,S14010,348.791262
5,S13001,348.470386


---
28. レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、上位5件を表示してください。

In [131]:
# df_receipt.groupby('store_cd')['amount'].apply(lambda x: np.median(x)).reset_index().sort_values('amount', ascending=False).head(5)
df_receipt.groupby('store_cd')['amount'].median().reset_index().sort_values('amount', ascending=False).head(5)

,store_cd,amount
28,S13052,190.0
30,S14010,188.0
51,S14050,185.0
44,S14040,180.0
7,S13003,180.0


---
29. レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示してください。

In [132]:
df_receipt.groupby('store_cd')['product_cd'].apply(lambda x: x.mode()).reset_index().sort_values('product_cd', ascending=False).head(10)

,store_cd,level_1,product_cd
29,S13051,2,P080804001
43,S14033,0,P071401001
24,S13041,0,P071401001
23,S13039,0,P071401001
37,S14023,0,P071401001
17,S13020,0,P071401001
16,S13019,0,P071401001
15,S13018,0,P071401001
12,S13015,0,P071401001
40,S14026,0,P071401001


---
30.  レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示してください。なお、分散の計算は、pandasのメソッドを使うこと。

In [133]:
tmp_df = df_receipt.groupby('store_cd')['amount'].var(ddof=1).reset_index()
tmp_df.rename(columns={'amount': 'amount_var'}).sort_values('amount_var', ascending=False).head(5)

,store_cd,amount_var
28,S13052,441863.252526
31,S14011,306442.242432
42,S14034,297068.392740
5,S13001,295558.842618
12,S13015,295427.197086


---
31. レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示してください。なお、標準偏差の計算は、pandasのメソッドを使うこと。

In [134]:
tmp_df = df_receipt.groupby('store_cd')['amount'].std(ddof=1).reset_index()
tmp_df.rename(columns={'amount': 'amount_std'}).sort_values('amount_std', ascending=False).head(5)

,store_cd,amount_std
28,S13052,664.727954
31,S14011,553.572256
42,S14034,545.039808
5,S13001,543.653237
12,S13015,543.532149


---
32.　レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めてください。

In [135]:
q = [i*25 for i in range(1, 5)]
# q = [25, 50, 75, 100]

np.percentile(df_receipt['amount'], q=q)

array([  102.,   170.,   288., 10925.])

---
33. レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出してください。

In [136]:
tmp_df = df_receipt.groupby('store_cd')['amount'].apply(lambda x: np.mean(x)).reset_index()
tmp_df = tmp_df.rename(columns={'amount': 'amount_mean'})
tmp_df.query('amount_mean >= 330')

,store_cd,amount_mean
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
34. レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めてください。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [137]:
df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id')['amount'].sum().mean()

2547.742234529256

---
35. レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示してください。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [144]:
tmp_df = df_receipt[~(df_receipt['customer_id'].str.startswith('Z'))]

tmp_df.groupby('customer_id')['amount'].sum().mean()
tmp_df[tmp_df['amount'] >= tmp_df.groupby('customer_id')['amount'].sum().mean()].reset_index()
# tmp_df[tmp_df['amount'] >= tmp_df.groupby('customer_id')['amount'].sum().mean()]

,index,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,89,20190615,1560556800,S13039,1102,2,CS039415000129,P090404061,1,2580
1,596,20181219,1545177600,S13015,1132,1,CS015515000083,P071301009,1,5280
2,1202,20190903,1567468800,S14011,1192,2,CS011415000006,P080104055,1,10925
3,2715,20180825,1535155200,S13035,1142,1,CS035415000125,P071301006,1,4760
4,2950,20180809,1533772800,S14023,1192,1,CS023513000144,P071401001,3,3300
...,...,...,...,...,...,...,...,...,...,...
310,101856,20181218,1545091200,S14025,1152,1,CS025415000188,P060204007,1,3000
311,102161,20190914,1568419200,S13018,1182,1,CS018605000002,P071301003,1,2650
312,102836,20170427,1493251200,S14006,1182,2,CS006415000147,P071301007,1,4880
313,102987,20170330,1490832000,S13002,1132,2,CS002515000078,P071301005,1,2680


In [138]:
df_amount_sum = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id')['amount'].sum()
amount_mean = df_amount_sum.mean()

df_amount_sum = df_amount_sum.reset_index()

df_amount_sum[df_amount_sum['amount'] >= amount_mean].head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


---
36. レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示してください。

In [139]:
tmp_df = pd.merge(df_receipt, df_store[['store_cd', 'store_name']], on='store_cd', how='inner')
tmp_df.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
37. 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示してください。

In [140]:
tmp_df = pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], on='category_small_cd', how='inner')
tmp_df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,4,401,40101,198.0,149.0,弁当類
1,P040101002,4,401,40101,218.0,164.0,弁当類
2,P040101003,4,401,40101,230.0,173.0,弁当類
3,P040101004,4,401,40101,248.0,186.0,弁当類
4,P040101005,4,401,40101,268.0,201.0,弁当類
5,P040101006,4,401,40101,298.0,224.0,弁当類
6,P040101007,4,401,40101,338.0,254.0,弁当類
7,P040101008,4,401,40101,420.0,315.0,弁当類
8,P040101009,4,401,40101,498.0,374.0,弁当類
9,P040101010,4,401,40101,580.0,435.0,弁当類


38. 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示してください。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [150]:
df_amount_sum = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

df_tmp = df_customer.loc[(df_customer['gender_cd'] == 1) & (~df_customer['customer_id'].str.startswith('Z'))]

pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0).head(10)

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
39. レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合してください。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [148]:
df_data = df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]

# 同日に売上のある顧客は削除（売上日数のカウントのため）
df_cnt = df_data[~df_data.duplicated(subset=['customer_id', 'sales_ymd'])] \
         .groupby('customer_id')['sales_ymd'].count().reset_index() \
         .sort_values('sales_ymd', ascending=False).head(20)

df_sum = df_data.groupby('customer_id')['amount'].sum().reset_index() \
         .sort_values('amount', ascending=False).head(20)

pd.merge(df_cnt, df_sum, how='outer', on='customer_id')

,customer_id,sales_ymd,amount
0,CS040214000008,23.0,NaN
1,CS015415000185,22.0,20153.0
2,CS010214000010,22.0,18585.0
3,CS010214000002,21.0,NaN
4,CS028415000007,21.0,19127.0
5,CS017415000097,20.0,23086.0
6,CS016415000141,20.0,18372.0
7,CS031414000051,19.0,19202.0
8,CS014214000023,19.0,NaN
9,CS022515000226,19.0,NaN


---
40. 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算してください。

In [143]:
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()

df_store_tmp['key'] = 0
df_product_tmp['key'] = 0

len(pd.merge(df_store_tmp, df_product_tmp, how='outer', on='key'))

531590